<h1> Segmenting and Clustering Neighborhoods in Toronto </h1>

<h2> IBM Data Science Professional Certificate </h2>

<h4> Capstone Week 3 Part 1 </h4>

In [1]:
import pandas as pd

In [2]:
website_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
table =pd.read_html(website_url)
table=table[0]
table.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
import numpy as np

table['Borough'].replace('Not assigned',np.nan,inplace=True)
table.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,Not assigned
1,M2A,NaN,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
# simply drop whole row with NaN in "Borough" column
table.dropna(subset=["Borough"], axis=0, inplace=True)

# reset index, because we droped rows
table.reset_index(drop=True, inplace=True)

table.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


<h2> Combine Postcode rows </h2>

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [6]:
### First method.

gb = table.groupby(('Postcode','Borough'))
table1 = gb['Neighbourhood'].unique()
table1.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  app.launch_new_instance()


Postcode  Borough    
M1B       Scarborough                            [Rouge, Malvern]
M1C       Scarborough    [Highland Creek, Rouge Hill, Port Union]
M1E       Scarborough         [Guildwood, Morningside, West Hill]
M1G       Scarborough                                    [Woburn]
M1H       Scarborough                                 [Cedarbrae]
Name: Neighbourhood, dtype: object

In [7]:
table1 = pd.DataFrame(data=table1)
table1.head()

,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,"[Rouge, Malvern]"
M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
M1G,Scarborough,[Woburn]
M1H,Scarborough,[Cedarbrae]


In [8]:
### Second method.

gb2=table.groupby(['Postcode','Borough'],as_index=False, sort=False).agg(','.join)
gb2['Neighbourhood']=set(gb2['Neighbourhood'])
gb2.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,York Mills West
1,M4A,North York,"Bedford Park,Lawrence Manor East"
2,M5A,Downtown Toronto,"Flemingdon Park,Don Mills South"
3,M6A,North York,Caledonia-Fairbanks
4,M7A,Queen's Park,"Humber Bay,King's Mill Park,Kingsway Park Sout..."


In [9]:
### Third method. Eventually I'll use this one.

table2= table.groupby(by=['Postcode','Borough']).agg(lambda x: ','.join(x))

table2.reset_index(level=['Postcode','Borough'], inplace=True)
table2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h2> Fill not assigned neighborhood with borough </h2>

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [10]:
table2['Neighbourhood'].replace('Not assigned',table2['Borough'],inplace=True)

table2.loc[table2['Postcode']=='M7A',]

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


<h2> Number of rows of my dataframe </h2>

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [11]:
print('My datafram has {} rows'.format(table2.shape[0]))

My datafram has 103 rows
